In [ ]:
import cv2 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from google.colab.patches import cv2_imshow
import random
import time

In [ ]:
def load_data(image):
  img = cv2.imread(image , 1)
  gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  return img , gray

In [ ]:
def rotate_image(image , angle , scale):
  # Rotating the image after Warp
  center = (image.shape[1]//2, image.shape[0]//2)
  rot_mat = cv2.getRotationMatrix2D( center, angle, scale )
  warp_rotate_dst = cv2.warpAffine(image, rot_mat, (image.shape[1], image.shape[0]))
  return warp_rotate_dst

In [ ]:
def create_rotated_images(image , gray):
  random_angle = random.randint(0,360)
  rotated_image = rotate_image(image , random_angle , 0.5)
  rotated_gray_image = rotate_image(gray , random_angle , 0.5)

  return rotated_image , rotated_gray_image , random_angle

In [ ]:
def operate_FREAK(image , gray):
  # Applying FREAK detector
  freak = cv2.xfeatures2d.FREAK_create()
  fast = cv2.xfeatures2d.SIFT_create()
  # Find the keypoints
  keypoints = fast.detect(gray, None)
  kp , des = freak.compute(gray, keypoints)

  # Marking the keypoint on the image using circles
  img=cv2.drawKeypoints(gray ,
                        kp ,
                        image ,
                        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
  
  return img , kp , des

In [ ]:
def find_matches(des_main , des_rotated):
  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks = 50)

  flann = cv2.FlannBasedMatcher(index_params, search_params)
  matches = flann.knnMatch(des_main,des_rotated,k=2)
  # store all the good matches as per Lowe's ratio test.
  good = []
  for m,n in matches:
      if m.distance < 0.5*n.distance:
          good.append(m)

  return good , len(matches)

In [ ]:
def operate(img):
  image , gray = load_data(img)

  rotated_image , rotated_gray_image , angle = create_rotated_images(image , gray)

  print("Rotate angle is :" , angle)
  cv2_imshow(rotated_image)


  start = time.time()
  KP_main_image , KP_main , des_main = operate_FREAK(image.copy() , gray.copy())
  KP_rotated_image, KP_rotated , des_rotated = operate_FREAK(rotated_image.copy() , rotated_gray_image.copy())
  des_main = des_main.astype('float32')
  des_rotated = des_rotated.astype('float32')



  good , matches = find_matches(des_main , des_rotated)

  MIN_MATCH_COUNT = 10
  if len(good)>MIN_MATCH_COUNT:
      src_pts = np.float32([ KP_main[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
      dst_pts = np.float32([ KP_rotated[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
      M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
      matchesMask = mask.ravel().tolist()
      h,w = rotated_gray_image.shape
      pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
      dst = cv2.perspectiveTransform(pts,M)
      img2 = cv2.polylines(rotated_gray_image,[np.int32(dst)],True,255,3, cv2.LINE_AA)
  else:
      print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
      matchesMask = None


  draw_params = dict(matchColor = (0,0,255), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
  
  img3 = cv2.drawMatches(gray,KP_main,rotated_gray_image,KP_rotated,good,None,**draw_params)
  end = time.time() - start
  cv2_imshow(img3)

  if len(good) > MIN_MATCH_COUNT:
    new_M, new_mask = cv2.findHomography(dst_pts,src_pts)
    out = cv2.warpPerspective(rotated_image,new_M,(rotated_image.shape[1], rotated_image.shape[0]),flags=cv2.INTER_LINEAR)
    cv2_imshow(out)

  print("Size of all matches :" , matches)
  print("Size of best matches : " , len(good))
  print("SIFT time : " , end)



In [ ]:
operate("GOW.jpg")

In [ ]:
operate("GOW.jpg")

In [ ]:
operate("GOW.jpg")